In [ ]:
import requests
from bs4 import BeautifulSoup
import re

URL = "https://www.washingtonpost.com/opinions/2023/07/24/musk-twitter-logo-x-rebrand/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
# print(page.text)

In [ ]:
# for cleaning text: This function accepts a text input, eliminates all non-alphanumeric characters, and outputs only letters and digits.
removelist = "."
clean = lambda s: str(re.sub('[\W_]+removelist+', ' ', s))

## Extract title of the article

In [ ]:
t=soup.find('span', attrs={ "data-qa": "headline-opinion-text"})

title=' '.join([clean(s.text) for s in t])
print(title)

Twitter removing the little blue bird is about more than a logo


## Extract date and time of when the article was published

In [ ]:
tt=soup.find('span', attrs={ "data-testid": "display-date"})

datetime=' '.join([clean(s.text) for s in tt])
print(datetime)

July 24 2023 at 5 33 p m EDT


## Extract article

In [ ]:
article = soup.body.findAll('article')
text = ' '.join([clean(s.text) for s in article[0].findAll('p')])
print(text)

Over the past few months, reports of Twitter’s death have been greatly exaggerated. Now, however, Elon Musk has made it official: Twitter — or, at least, “Twitter” — is no more. Let us mourn the cheery little blue bird. The microblogging platform owner and Tesla CEO announced this weekend that Twitter would rebrand to “X.” The gambit is part of Mr. Musk’s long-standing desire to build what he has called an “everything app” — with communications, payments and any other functionality a consumer might desire all part of the package, much like Tencent’s WeChat in China or Grab in Southeast Asia. Or as Twitter chief executive Linda Yaccarino tweeted  (if we may still refer to it as “tweeting”), “X is the future state of unlimited interactivity.” We’ll see how Mr. Musk, having driven away many of the advertisers that bolstered Twitter’s value, gets along after annihilating another of the platform’s most valuable assets: its brand. In any case, the moment is somber. Twitter today is already w

In [ ]:
# Remove 'About the Editorial Board'
r='Editorials represent '

head, sep, tail = text.partition(r)

print(head)

Over the past few months, reports of Twitter’s death have been greatly exaggerated. Now, however, Elon Musk has made it official: Twitter — or, at least, “Twitter” — is no more. Let us mourn the cheery little blue bird. The microblogging platform owner and Tesla CEO announced this weekend that Twitter would rebrand to “X.” The gambit is part of Mr. Musk’s long-standing desire to build what he has called an “everything app” — with communications, payments and any other functionality a consumer might desire all part of the package, much like Tencent’s WeChat in China or Grab in Southeast Asia. Or as Twitter chief executive Linda Yaccarino tweeted  (if we may still refer to it as “tweeting”), “X is the future state of unlimited interactivity.” We’ll see how Mr. Musk, having driven away many of the advertisers that bolstered Twitter’s value, gets along after annihilating another of the platform’s most valuable assets: its brand. In any case, the moment is somber. Twitter today is already w

## Save data

In [ ]:
[title, datetime, head]

['Twitter removing the little blue bird is about more than a logo',
 'July 24 2023 at 5 33 p m EDT',
 'Over the past few months, reports of Twitter’s death have been greatly exaggerated. Now, however, Elon Musk has made it official: Twitter — or, at least, “Twitter” — is no more. Let us mourn the cheery little blue bird. The microblogging platform owner and Tesla CEO announced this weekend that Twitter would rebrand to “X.” The gambit is part of Mr. Musk’s long-standing desire to build what he has called an “everything app” — with communications, payments and any other functionality a consumer might desire all part of the package, much like Tencent’s WeChat in China or Grab in Southeast Asia. Or as Twitter chief executive Linda Yaccarino tweeted  (if we may still refer to it as “tweeting”), “X is the future state of unlimited interactivity.” We’ll see how Mr. Musk, having driven away many of the advertisers that bolstered Twitter’s value, gets along after annihilating another of the pl

In [ ]:
import pandas as pd
df=pd.DataFrame([[title, datetime, head]],  columns=['title', 'datetime', 'text'])
# df.to_csv('./washingtonpost.csv', index=False)

## Sentiment analysis

In [ ]:
pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch


In [ ]:
# Replace 'roberta-base' with the desired pre-trained model (e.g., 'roberta-large' for a larger model)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_long_text(text, max_length=512):
    # Split the long text into smaller chunks and encode them separately
    chunks = [text[i:i + max_length] for i in range(0, len(text), max_length)]
    input_ids = []
    attention_masks = []

    for chunk in chunks:
        encoded_dict = tokenizer.encode_plus(
            chunk,

            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)


In [ ]:
def analyze_sentiment(text):
    # Preprocess the long text
    input_ids, attention_masks = preprocess_long_text(text)

    # Set the model to evaluation mode
    model.eval()

    # Make predictions
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_masks)

    # Get the predicted probabilities for each class (negative, neutral, positive)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

    # Determine the overall sentiment based on the class with the highest probability
    sentiment_label = torch.argmax(probabilities[0]).item()
    sentiment_score = probabilities[0][sentiment_label].item()

    # Convert the sentiment label to a human-readable format
    sentiment_classes = ['Negative', 'Neutral', 'Positive']
    predicted_sentiment = sentiment_classes[sentiment_label]

    return predicted_sentiment, sentiment_score


In [ ]:
predicted_sentiment, sentiment_score = analyze_sentiment(head)
print(f"Predicted Sentiment: {predicted_sentiment}")
print(f"Sentiment Score: {sentiment_score}")


Predicted Sentiment: Negative
Sentiment Score: 0.5112683773040771
